# **Proyecto Final Ciencia De Datos**
## **Integrantes**:

## John Sebastián Galindo Hernández
## Miguel Ángel Moreno Beltrán

## **Universidad de Cundinamarca**
---
### **Objetivo**

##### Realizar un dashboard acerca de las vulnerabilidades que pueden presenar los sofwares instalados en las maquinas de los laboratorios de sistemas en la universidad de cundinamarca.


### **Estructura JSON**

#### Para el proyecto se considera manejar 3 colecciones (contendores) diferentes para mantener el codigo legible, sencillo y que permita el buen rendimiento.

Para cada software se obtendrá la información de los ultimos 10 CVEs registrados.

#### **1. Coleccion para la maquina**

**Nombre de la Colección:** Machines

Esta colección almacenará toda la información relacionada con la maquina a la que se le van a a escanear los softwares. Entre los datos que se consideraron relevanes a guardar están:
+ **id:** dirección mac de la maquina
+ **so:** Sistema operativo de la maquina
+ **soVersion:** Version del sistema operativo
+ **scanDate:** Fecha del ultimo escaneo hecho
+ **softwares:** Lista sobre los softwares que fueron encontrados en el escaneo

In [11]:
#EJEMPLO
{
    "id": "00:1A:2B:3C:4D:5E",
    "so": "Windows",
    "soVersion": "Windows 10 pro",
    "scanDate": "19-04-2024",
    "softwares": ["Excel", "Powerpoint", "SQLServer", "Cisco Packet Tracer"]
}

{'id': '00:1A:2B:3C:4D:5E',
 'so': 'Windows',
 'soVersion': 'Windows 10 pro',
 'scanDate': '19-04-2024',
 'softwares': ['Excel', 'Powerpoint', 'SQLServer', 'Cisco Packet Tracer']}

#### **2. Coleccion para los softwares**

**Nombre de la Colección:** Softwares

Esta colección almacenará toda la información relacionada con los softwares detectados en las maquinas. Entre los datos que se consideraron relevanes a guardar están:
+ **id:** identificador unico (name+vendor)
+ **name:** Nombre del software
+ **vendor:** Nombre de la empresa que produce el software
+ **scanDate:** Fecha del ultimo escaneo hecho
+ **associatedMachines**: lista de las maquinas que tienen este software en su lista de softwares encontrados
+ **cves:** Lista de los ids de las vulnerabilidades que fueron encontrados en la consulta del software

In [ ]:
#EJEMPLO
{
    "id": "ExcelMicrosoft",
    "name": "Excel",
    "vendor": "Microsoft",
    "scanDate": "20-04-2024",
    "asociatedMachines": ["00:1A:2B:3C:4D:5E","00:1A:2F:3C:8D:5E"],
    "cves": ["CVE-2024-1233", "CVE-2019-0708"]
}

#### **3. Coleccion para los CVEs**

**Nombre de la Colección:** Cves

Esta colección almacenará toda la información relacionada con las vulnerabilidades obtenidas por software. Entre los datos que se consideraron relevanes a guardar están:
+ **CVE_ID**: Identificador del cve
+ **description:** Descripcion de la vulnerabilidad
+ **vulnStatus**: Estado que tiene el cve, ayuda a identificar si ya fue analizado o esta en proceso
+ **metrics**: Es un JSON con informacion acerca de las metricas cvss
    + **baseScore**: Es el puntaje resultante del cvss
    + **baseSeverity**: La serveridad que se le da a la vulnerabilidad en base del puntaje base
    + **exploitabilitiScore**: El puntaje de la probabilidad con la que se puede explotar el software
    + **impactScore**: Puntaje acerca de el impacto que puede generar esta vulnerabilidad en la maquina
    + **cvssDetailedData**: Es un Json anidado que contiene toda la informacion proporcionada por el analisis cvss tales como la version de ese cvss, el string donde se resume toda las mediciones de los diferentes aspectos de la vulnerabilidad, y el dato especifico para cada parte escaneada en el cvss como la complejidad, los privilejgios requeridos, entre otros.
+ **AttackDetails**: Es un JSON anidado con detalles sobre el ataque que se realiza con esa vulnerabilidad, estos se identifican con 1 si estan presentes o 0 si no tienen relacion con ese CVE, entre algunos de estos detalles esta el si es una corrupcion en la memoria, si es una inyección sql, una vulnerabilidad relacionada con ganar privilegios, entre otros.

In [ ]:
 {
    "CVE_ID": "CVE-2023-3297",
    "description": "In Ubuntu's accountsservice an unprivileged local attacker can trigger a use-after-free vulnerability in accountsservice by sending a D-Bus message to the accounts-daemon process.",
    "vulnStatus": "Analyzed",
      "metrics": {
        "baseScore": 7.8 ,
        "baseSeverity": "HIGH",
        "exploitabilityScore": 1.8,
        "impactScore": 5.9,
        "cvssDetailedData": {
          "version": "3.1",
          "vectorString": "CVSS:3.1/AV:L/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H",
          "attackVector": "LOCAL",
          "attackComplexity": "LOW",
          "privilegesRequired": "LOW",
          "userInteraction": "NONE",
          "scope": "UNCHANGED",
          "confidentialityImpact": "HIGH",
          "integrityImpact": "HIGH",
          "availabilityImpact": "HIGH"
        }
      },
      "AttackDetails": {
        "isInCISAKEV": "0",
        "isOverflow": "0",
        "isMemoryCorruption": "1",
        "isSqlInjection": "0",
        "isXss": "0",
        "isDirectoryTraversal": "0",
        "isFileInclusion": "0",
        "isCsrf": "0",
        "isXxe": "0",
        "isSsrf": "0",
        "isOpenRedirect": "0",
        "isInputValidation": "0",
        "isCodeExecution": "0",
        "isBypassSomething": "0",
        "isGainPrivilege": "0",
        "isDenialOfService": "0",
        "isInformationLeak": "0",
        "isUsedForRansomware": "0"
      }
    }

### **Consulta de APIs**

Se consultaran 2 Apis principalmente:
1. **CVE Details**: Se usará para conocer la información de un software y de las vulnerabilidades relacionadas a este.
   
   **Beneficios**: Permite complementar también la información de los cves con información detallada del ataque. Además gracias al funcionamieno de sus creditos por tiempo permite la automatización del proceso de consulta.
   
   **Costos:** Gratis con aproximadamente 5 consultas por minuto.
   
2. **Nist**: Se usará para conocer la información especifica de un CVE, permite la consulta con el id del CVE y brinda información relevane como la descripcion y todos los datos relacionados al puntaje cvss.

   **Beneficios**: Tiene una facil consulta, permite crear suscripción de estudiante, tiene información clave que minimiza el codigo como la criticidd y la información especifica de cada parte tenida en cuenta en el análisis cvss.
   
   **Costos:** Suscripción de estudiante con correo institucional que permite 50 consultas por minuto ó Gratis con 5 consultas por minuto.

### **Graficas a Realizar**

* Top vulnerabilidades
* Top Software vulnerable
* Top Cves by machine
* Top software con los CVEs mas recientes
* Maquinas con mas impact score
* Tipo de ataque mas presentado
* Torta por porcentaje de ataque de complejidad.
* Torta por porcentaje por interaccion con el usuario.
* Torta por porcentaje por privilegios requeridos